In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Flatten
from scipy.interpolate import CubicSpline
from sklearn.preprocessing import LabelEncoder

In [2]:
all_data = pd.read_csv("spline-test.csv", index_col=2)
ct_data = all_data['CT'].values #drop the /N values instead
age_data = all_data['PatientAge'].values
print(ct_data)

[23.95 33.65 27.28 31.22 26.8  17.34 31.08 29.66 24.8  32.4  23.6  28.4 ]


In [3]:
X = pd.DataFrame()
X['age'] = age_data
X['ct'] = ct_data
print(X)

          age     ct
0   15.000000  23.95
1   17.000000  33.65
2    8.000000  27.28
3    7.000000  31.22
4    0.666667  26.80
5    6.000000  17.34
6    8.000000  31.08
7   15.000000  29.66
8    5.000000  24.80
9   10.000000  32.40
10  10.000000  23.60
11   6.000000  28.40


In [4]:
def cubic(u):
    t = np.maximum(0, u)
    return t * t * t
    
# Generate cubic spline basis functions
def cubic_spline_basis_functions(x, knots):
    k0 = knots[0]
    k1 = knots[1]
    k2 = knots[2]
    c = (k2 - k0) * (k2 - k0)
    spline_values = ((cubic(x - k0) - cubic(x - k1) * (k2 - k0)/(k2 - k1) + cubic(x - k2) * (k1 - k0)/(k2 - k1)) / c )
    
    # spline_functions = [CubicSpline(knots, np.eye(len(knots))[i], extrapolate=False) for i in range(len(knots))]
    # spline_values = np.column_stack([f(x) for f in spline_functions])
    # print("x", x)
    # print("knots", knots)
    # print("splines", spline_values)
    # print("*********")
    return spline_values

In [5]:
# Knots for restricted cubic splines
age_knots = [2, 10, 16]
ct_knots = [18.6, 25.2, 34.5]

# Apply cubic spline transformations to variables
age_spline = cubic_spline_basis_functions(age_data, age_knots)
ct_spline = cubic_spline_basis_functions(ct_data, ct_knots)

# print(age_spline)
# print(ct_spline)

In [10]:
# Predefined weights and bias
weights = np.array([[-0.3806], [0.3034], [-0.2139], [0.1545]])

data = np.column_stack((age_data, age_spline, ct_data, ct_spline))

bias = np.array([0]).reshape(1, )

bias_and_weights = []
bias_and_weights.append(weights)
bias_and_weights.append(bias)

print("wanted shape:", weights.shape)

In [11]:
print(data.shape)
print(weights.shape)

(12, 4)
(4, 1)


In [12]:
# Create TensorFlow model
model = tf.keras.Sequential()
model.add(Flatten(input_shape=weights.shape))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss = "binary_crossentropy")

print(model.get_weights())
model.set_weights(bias_and_weights)
print(model.get_weights())

model.intercept = 6.5164
model.summary()

print("number of tensors:", len(model.inputs))

pred = model.predict(data)
pred

wanted shape: (4, 1)
[array([[ 0.8658608 ],
       [-0.19740063],
       [-1.0420867 ],
       [-0.7041507 ]], dtype=float32), array([0.], dtype=float32)]
[array([[-0.3806],
       [ 0.3034],
       [-0.2139],
       [ 0.1545]], dtype=float32), array([0.], dtype=float32)]
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 4)                 0         
                                                                 
 dense_1 (Dense)             (None, 1)                 5         
                                                                 
Total params: 5 (20.00 Byte)
Trainable params: 5 (20.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
number of tensors: 1
1/1 [==============================] - 0s 21ms/step


array([[0.00041412],
       [0.00026712],
       [0.00028711],
       [0.00028915],
       [0.00349272],
       [0.00274957],
       [0.00022868],
       [0.00023171],
       [0.00089278],
       [0.00016201],
       [0.00034038],
       [0.0004444 ]], dtype=float32)